In [ ]:
!pip install langchain==0.1.14 langchain-google-genai==0.0.11 google-generativeai


In [ ]:
#import the function
import os
import re
import gradio as gr
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.memory import ConversationBufferWindowMemory

In [ ]:
# Configure Gemini LLM

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    convert_system_message_to_human=True,
    api_key=GOOGLE_API_KEY
)


In [ ]:
# Define Tools

def extract_numbers(text):
    return [float(x) for x in re.findall(r"-?\d+\.?\d*", text)]

def add_numbers(text: str):
    nums = extract_numbers(text)
    if not nums:
        return "No numbers found."
    return f"The sum of {', '.join(map(str, nums))} is {sum(nums)}"

def multiply_numbers(text: str):
    nums = extract_numbers(text)
    if not nums:
        return "No numbers found."
    result = 1
    for n in nums:
        result *= n
    return f"The product of {', '.join(map(str, nums))} is {result}"

tools = [
    Tool(name="add_numbers", func=add_numbers, description="Use to add numbers"),
    Tool(name="multiply_numbers", func=multiply_numbers, description="Use to multiply numbers"),
]

In [ ]:
# Conversational Memory

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=100, return_messages=True)

# Initialize LangChain Agent

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=False
)

In [ ]:
# Gradio Function with Tool Tracking

conversation_history = []
last_tool_used = None

def chat_gradio(user_message, clear=False):
    global conversation_history, last_tool_used
    if clear:
        conversation_history = []
        memory.clear()
        last_tool_used = None
        return ""

    # Special handling for "Which tool was used?"
    if "which tool" in user_message.lower():
        if last_tool_used:
            response_text = f"I used the tool '{last_tool_used}' for the previous calculation."
        else:
            response_text = "I did not use any tools for the previous question."
    else:
        # Let the agent handle tool routing automatically
        response_text = agent.run(user_message)

        # Check if agent used a tool (simple detection)
        last_tool_used = None
        for tool in tools:
            if tool.name.replace("_", " ") in response_text.lower() or "sum" in response_text.lower() or "product" in response_text.lower():
                last_tool_used = tool.name
                break

    # Save conversation
    conversation_history.append(("You", user_message))
    conversation_history.append(("Result", response_text))

    chat_text = ""
    for speaker, message in conversation_history:
        chat_text += f"{speaker}: {message}\n\n"

    return chat_text

In [ ]:
# Launch Gradio

iface = gr.Interface(
    fn=chat_gradio,
    inputs=[gr.Textbox(lines=2, placeholder="Type your message here..."),
            gr.Checkbox(label="Clear Conversation")],
    outputs=gr.Markdown(),
    title="LangChain Agent with Tools",
    description="A LangChain agent that automatically decides which tool to use and tracks tool usage.",
    allow_flagging="never"
)

iface.launch()


/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://58a97a55e2ba108222.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
